In [1]:
import autograd.numpy as np
import copy
from itertools import product
import numpy.testing as np_test
import Parameters
from Parameters import \
    VectorParam, ScalarParam, PosDefMatrixParam, ModelParamsDict
import unittest

from autograd.util import quick_grad_check
from autograd import grad, jacobian, hessian

In [13]:
k = 2
mat = np.full(k ** 2, 0.2).reshape(k, k) + np.eye(k)

lb = -0.1
ub = 5.2
val = 0.5 * (ub - lb) + lb
vec = np.linspace(lb, ub, k)

vp_scalar = ScalarParam('scalar', lb=lb - 0.1, ub=ub + 0.1)
vp_mat = PosDefMatrixParam('matrix', k)
vp_vec = VectorParam('vector', k, lb=lb - 0.1, ub=ub + 0.1)

vp_scalar.set(val)
vp_vec.set(vec)
vp_mat.set(mat)

mp = ModelParamsDict()
mp.push_param(vp_scalar)
mp.push_param(vp_vec)
mp.push_param(vp_mat)

# To take advantage of quick_grad_check(), define scalar functions of
# each parameter.
def ScalarFun(val_free):
    vp_scalar_ad = copy.deepcopy(vp_scalar)
    vp_scalar_ad.set_free(val_free)
    return vp_scalar_ad.get()

def VecFun(val_free):
    vp_vec_ad = copy.deepcopy(vp_vec)
    vp_vec_ad.set_free(val_free)
    return np.linalg.norm(vp_vec_ad.get())

def MatFun(val_free):
    vp_mat_ad = copy.deepcopy(vp_mat)
    vp_mat_ad.set_free(val_free)
    return np.linalg.norm(vp_mat_ad.get())

def ParamsFun(val_free):
    mp_ad = copy.deepcopy(mp)
    mp_ad.set_free(val_free)
    return mp_ad['scalar'].get() + \
           np.linalg.norm(mp_ad['vector'].get()) + \
           np.linalg.norm(mp_ad['matrix'].get())


quick_grad_check(ScalarFun, vp_scalar.get_free())
quick_grad_check(VecFun, vp_vec.get_free())
quick_grad_check(MatFun, vp_mat.get_free())
quick_grad_check(ParamsFun, mp.get_free())

Checking gradient of <function ScalarFun at 0x7f933682ef50> at 0.0
Gradient projection OK (numeric grad: -1.37499999972, analytic grad: -1.375)
Checking gradient of <function VecFun at 0x7f933682ed70> at [-3.98898405  3.98898405]
Gradient projection OK (numeric grad: 0.056986754462, analytic grad: 0.0569867544487)
Checking gradient of <function MatFun at 0x7f933682e5f0> at [ 1.09544512  0.18257419  1.08012345]
Gradient projection OK (numeric grad: -0.0127102872982, analytic grad: -0.0127102877958)
Checking gradient of <function ParamsFun at 0x7f933682e8c0> at [ 0.         -3.98898405  3.98898405  1.09544512  0.18257419  1.08012345]
Gradient projection OK (numeric grad: 0.219365717591, analytic grad: 0.219365717742)
